In [47]:
import importlib
import sys

import tqdm

# Reload the module to ensure the latest version is used
importlib.reload(sys.modules['src.orca_hls_utils.s3_utils'])
from src.orca_hls_utils.s3_utils import get_all_folders, verify_playlist

# Define the S3 bucket and the prefix for the hydrophone (e.g. rpi_orcasound_lab)
BUCKET = "audio-orcasound-net"
PREFIX = "rpi_orcasound_lab/hls/"

# Get all timestamp folders using s3_utils.get_all_folders
folders = get_all_folders(BUCKET, PREFIX)
print(f"Found {len(folders)} folders under prefix: {PREFIX}")
report = []
if not folders:
    print(f"No folders found under prefix: {PREFIX}")
else:
    for folder in tqdm.tqdm(folders[10000:11000]):  
        folder_prefix = PREFIX + folder + "/"  # e.g. "rpi_orcasound_lab/hls/1541061134/"
        res = verify_playlist(BUCKET, folder_prefix)
        if res:
            report.append({'folder': folder} | res)

Found 16246 folders under prefix: rpi_orcasound_lab/hls/


100%|██████████| 1000/1000 [23:55<00:00,  1.44s/it]


In [48]:
import pandas as pd
df = pd.DataFrame(report)
df.to_csv("playlist_report.csv", index=False)

In [61]:
display(df)
# change missing_files column to store a list of missing file numbers instead of a string

,folder,missing_files,extra_files,len_extra,timeStamp,len_missing
0,1583937033,[],"[351, 352, 353, 354, 355, 356, 357, 358]",8,2020-03-11 14:30:33,0
1,1583940633,[],"[352, 353, 354, 355, 356, 357, 358]",7,2020-03-11 15:30:33,0
2,1583944234,[],"[338, 339, 340, 341, 342, 343, 344]",7,2020-03-11 16:30:34,0
3,1583947043,[],"[044, 045, 046, 047, 048, 049, 050]",7,2020-03-11 17:17:23,0
4,1584019819,[2158],[],0,2020-03-12 13:30:19,1
...,...,...,...,...,...,...
568,1605364219,[2158],[],0,2020-11-14 14:30:19,1
569,1605774619,[2158],[],0,2020-11-19 08:30:19,1
570,1606314629,[2157],[],0,2020-11-25 14:30:29,1
571,1606336220,[2158],[],0,2020-11-25 20:30:20,1


In [58]:
df['timeStamp'] = pd.to_datetime(df['folder'], unit='s')

C:\Users\stlp\AppData\Local\Temp\ipykernel_13164\734088281.py:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timeStamp'] = pd.to_datetime(df['folder'], unit='s')


In [57]:
df['len_extra'].value_counts()

len_extra
0    569
7      3
8      1
Name: count, dtype: int64

In [52]:
df['extra_files'].iloc[0]

['351', '353', '355', '356', '358', '352', '354', '357']

In [60]:
df['extra_files'] = df['extra_files'].apply(lambda x: sorted(x))
df['len_extra'] = df['extra_files'].apply(lambda x: len(x))
df['len_missing'] = df['missing_files'].apply(lambda x: len(x))